## Criando solução com RAG

In [2]:
# Separar o documento em chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Embeddings gratuitos via Hugging Face
from langchain.embeddings import HuggingFaceEmbeddings
# LLM gratuito via Hugging Face
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
# Banco de dados vetorial
from langchain_community.vectorstores import Chroma
# Carregar arquivos PDF
from langchain_community.document_loaders import PyPDFLoader
# Prompt
from langchain.chains.question_answering import load_qa_chain

c:\Users\Deyvid\ai-for-dev-rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
## Chave API
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

Load dos modelos (Embeddings e LLM)

In [5]:
# Carregar tokenizer e modelo
model_name = "sshleifer/tiny-gpt2"  # ou outro modelo de sua escolha
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Criar pipeline
hf_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.7,
    do_sample=True,
)

# Integrar com LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu
C:\Users\Deyvid\AppData\Local\Temp\ipykernel_25320\3812263793.py:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


Carregar o arquivo (PDF)

In [6]:
pdf_link = "datasets/remuneracao_docentes_UF_2020.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

Separar em Chunks (separar o conteúdo do arquivo em pequenas partes)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 4000,
  chunk_overlap = 20,
  length_function = len,
  add_start_index = True,
)

chunks = text_splitter.split_documents(pages)

Salvar no Vector DB

In [8]:
# Criar modelo de embeddings
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")
db.persist()

C:\Users\Deyvid\AppData\Local\Temp\ipykernel_25320\1547133016.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Deyvid\ai-for-dev-rag\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Deyvid\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disa